In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [ ]:
dataStocks = pd.read_csv("../dataset_raw/historical_stocks.csv")

In [ ]:
# si controlla se ci sono valori nulli
dataStocks.isnull().sum()

In [ ]:
# si controlla se ci sono aziende con lo stesso nome che compaiono più volte
dataStocks.groupby('name').count()

In [ ]:
# esempio di un'azienda che compare due volte, ma una volta non cha il settore e l'industria specificati
dataStocks[dataStocks.name == 'GASLOG LP.']

In [ ]:
app = dataStocks.groupby('name').count()[['ticker', 'sector']]
app

In [ ]:
# il numero di volte che l'azienda compare (determinato da ticker) è diverso dal numero dei nomi dati al settore.
# Questo significa che l'azienda per sbaglio non ha messo sempre il nome del settore. Prendo le aziende che non
# hanno messo sempre il nome al settore
app2 = app.loc[app.ticker != app.sector]
# prendo le aziende che hanno messo almeno un nome nel settore (le aziende con non l'hanno mai messo vanno buttate)
app3 = app2.loc[app2.sector != 0]
# lista delle aziende che si sono scordate di mettere il nome del settore, ma che l'hanno messo almeno una volta
lista_aziende_con_NaN = list(app3.index)

In [ ]:
# set delle aziende che si sono scordate di mettere il nome del settore, ma l'hanno messo almeno una volta
set_aziende_con_NaN = set(lista_aziende_con_NaN)
set_aziende_con_NaN

In [ ]:
diz = {}
for index, row in dataStocks.iterrows():
    # se il nome dell'azienda si trova tra quelle che non hanno assegnato tutti i nomi a sector e industry
    if row['name'] in set_aziende_con_NaN:
        # e se tutti i valori della riga non sono nulli (quindi ho l'azienda pippo che compare 5 volte e 1 
        # volta mette tutti i valori giusti, io devo vedere in quale riga mette tutti quei valori giusti, 
        # in modo da prendere le informazioni) 
        # (un po' brutto farlo così)
        # inoltre se l'azienda pippo compare 5 volte e 2 volte mette tutti i volori giusti, entrembe le volte questi
        # vengono messi nel dizionario, penso che è come se c'è una sovrascrittura tutte le volte e non
        # è un problema perché i valori sono sempre uguali
        if not row.isnull().values.any():
            # print("Nome:   " + row['name'] + "  ", "Settore: " + row['sector'], "Industria:   " + row['industry'])
            # alla chiave nome dell'azienda, associo la lista di valori corretti
            diz[row['name']] = [row['sector'], row['industry']]

In [ ]:
# per tutte le chiavi della lista, quindi per tutte le aziende che si sono scordate qualcosa
for key in diz.keys():
    # sostituisco nel dataset tutti i valori in sector e industry con i valori corretti
    # sostituisco anche quelli che già ci sono, ma non fa niente
    dataStocks.loc[dataStocks.name == key, 'sector'] = diz[key][0]
    dataStocks.loc[dataStocks.name == key, 'industry'] = diz[key][1]

In [ ]:
# 6460 - 1018 = 5442
dataStocks.dropna(axis=0, inplace=True)

In [ ]:
# si sostituiscono li virgole che compaiono nel nome, in quanto potrebbero dare problemi quando 
# si processano i dati
dataStocks['name'] = dataStocks['name'].str.replace(',','')

In [ ]:
dataStocks.to_csv('../dataset/historical_stocks_clean.csv', index = False, header = False)